자산배분은 상관관계가 낮거나 반대 방향인 우상향하는 자산군에 분산 투자하는 것입니다. 그러므로, 자산군 과거 수익률에서 상관관계를 체크해야 합니다. 대표적으로 주식, 국채, 물가채, 회사채, 금, 원자재, 부동산(리츠) 정도가 많이 다뤄지므로 이들의 상관관계를 보겠습니다.

In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import backtrader as bt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pyfolio as pf
import quantstats
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)

C:\ProgramData\Anaconda3\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


VTSMX (미국 주식), VTMGX (선진국 주식), VEIEX (신흥국 주식), PGBIX (전세계 채권), VFITX (미국 중기 국채), VUSTX (미국 장기 국채), LQD (미국 회사채), TIP (미국 물가채), IAU (금), GSG (원자재), VGSIX (부동산 - 리츠)를 사용하여 체크하겠습니다. 기간은 GSG 데이터가 2006년 7월 21부터 구해지므로, 2006. 08. 01 ~ 2021. 02. 12로 하겠습니다.

In [2]:
start = '2006-08-01'
end = '2021-02-12'

vtsmx = web.DataReader("VTSMX", 'yahoo', start, end)['Adj Close'].to_frame("vtsmx")
vtmgx = web.DataReader("VTMGX", 'yahoo', start, end)['Adj Close'].to_frame("vtmgx")
veiex = web.DataReader("VEIEX", 'yahoo', start, end)['Adj Close'].to_frame("veiex")
pgbix = web.DataReader("PGBIX", 'yahoo', start, end)['Adj Close'].to_frame("pgbix")
vfitx = web.DataReader("VFITX", 'yahoo', start, end)['Adj Close'].to_frame("vfitx")
vustx = web.DataReader("VUSTX", 'yahoo', start, end)['Adj Close'].to_frame("vustx")
lqd = web.DataReader("LQD", 'yahoo', start, end)['Adj Close'].to_frame("lqd")
tip = web.DataReader("TIP", 'yahoo', start, end)['Adj Close'].to_frame("tip")
iau = web.DataReader("IAU", 'yahoo', start, end)['Adj Close'].to_frame("iau")
gsg = web.DataReader("GSG", 'yahoo', start, end)['Adj Close'].to_frame("gsg")
vgsix = web.DataReader("VGSIX", 'yahoo', start, end)['Adj Close'].to_frame("vgsix")

In [3]:
df_list = [vtsmx, vtmgx, veiex, pgbix, vfitx, vustx, lqd, tip, iau, gsg, vgsix]
ticker_list = ['vtsmx', 'vtmgx', 'veiex', 'pgbix', 'vfitx', 'vustx', 'lqd', 'tip', 'iau', 'gsg', 'vgsix']

In [4]:
return_list = []

for df in df_list:
    df_return_list = []
    
    for i in range(len(df)-1):
        return_cal = (df.iloc[i+1,0]-df.iloc[i,0])/df.iloc[i,0]
        df_return_list.append(return_cal)
    
    # 맨 앞 날짜는 값 없음
    df_return_list.insert(0, np.nan)
    
    return_list.append(df_return_list)

In [5]:
return_dict = {}

for i in range(len(ticker_list)):
    return_dict[ticker_list[i]] = return_list[i]

In [6]:
return_df = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in return_dict.items() ]))

In [8]:
return_df.corr(method = 'pearson')

,vtsmx,vtmgx,veiex,pgbix,vfitx,vustx,lqd,tip,iau,gsg,vgsix
vtsmx,1.000000,0.893402,0.842409,-0.127828,-0.388724,-0.417488,0.140530,-0.200674,0.033117,0.450034,0.787547
vtmgx,0.893402,1.000000,0.885186,-0.126535,-0.332871,-0.374225,0.170749,-0.123178,0.135953,0.490766,0.684306
veiex,0.842409,0.885186,1.000000,-0.118212,-0.340970,-0.353041,0.164499,-0.129440,0.131034,0.500417,0.650429
pgbix,-0.127828,-0.126535,-0.118212,1.000000,0.486672,0.461462,0.352464,0.449804,0.100816,-0.084958,-0.096076
vfitx,-0.388724,-0.332871,-0.340970,0.486672,1.000000,0.835349,0.380042,0.684878,0.183520,-0.210122,-0.235033
vustx,-0.417488,-0.374225,-0.353041,0.461462,0.835349,1.000000,0.435645,0.676696,0.167712,-0.262773,-0.231023
lqd,0.140530,0.170749,0.164499,0.352464,0.380042,0.435645,1.000000,0.454938,0.119341,0.075980,0.115284
tip,-0.200674,-0.123178,-0.129440,0.449804,0.684878,0.676696,0.454938,1.000000,0.248624,0.016008,-0.131733
iau,0.033117,0.135953,0.131034,0.100816,0.183520,0.167712,0.119341,0.248624,1.000000,0.265323,0.026426
gsg,0.450034,0.490766,0.500417,-0.084958,-0.210122,-0.262773,0.075980,0.016008,0.265323,1.000000,0.297317


미국 주식(VTSMX), 선진국 주식(VTMGX), 신흥국 주식(VEIEX)는 상관계수가 0.8 이상으로 크기 때문에 각자가 자산군 수준에서 분산 효과를 내기보다는 주식이라는 자산군 안에서 지역 분산을 위해 적절한 비중으로 나누는 것을 생각할 수 있습니다. 전세계 채권(PGBIX), 미국 중기 국채(VFITX), 미국 장기 국채(VUSTX) 세 가지는 주식과 상관관계가 작고, 특히 전세계 채권과 미국 국채 상관계수도 0.487로 어느 정도 분산 효과가 있어 보입니다. 미국 회사채(LQD)는 포함된 다른 모든 자산군과의 상관계수가 0.5 이하로 분산 효과가 좋습니다. 미국 물가채(TIP)는 미국 국채와 상관계수가 높으나 금리 인상 등에 대비하여 추가합니다. 금(IAU)과 원자재(GSG)는 인플레이션 대비 목적이 있고, 서로 상관관계가 낮은 편입니다. 부동산(VGSIX)은 리츠 회사 주식으로 이루어져 있어서인지 적당한 수준의 인플레이션에 강하다고 알려져 있으나 주식 시장과 상관관계가 매우 높은 편입니다.